In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az
import time
import re

from bs4 import BeautifulSoup
import requests

In [9]:
url = 'https://www.sports-reference.com/cbb/postseason/men/2024-ncaa.html'
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
else:
    print(f"Failed to retrieve content, status code: {response.status_code}")

# List of 64 teams in the playoffs
a_tags = soup.find_all('a')[67:131]

# Parse HTML
base_url = "https://www.sports-reference.com"
urls = [base_url + tag['href'] for tag in a_tags if 'href' in tag.attrs]

df_list = []
counter = 0

for team in urls:
    print('///// Currently parsing tables for : {} /////'.format(team.split('/')[5].upper()))
    print('\n')
    year = int(urls[0].split('/')[-1].split('.')[0])
    lookback_years = (lambda x: [x - i for i in range(15)])(year)


    for yr in lookback_years:

        new_team_url = (lambda url, year: re.sub(r'(\d{4})(\.html)$', str(year) + r'\2', url))(team, yr)
        print("Fetching data from {} ... ".format(yr))
        res = requests.get(url=new_team_url)
        time.sleep(5)
           
        if counter == 15:
            print('Reaching requests limit, resetting counter')
            time.sleep(65)
            counter = 0

        if res.status_code == 200:
            soup = BeautifulSoup(res.text, 'html.parser')
            table_container = soup.select_one('#div_per_game')
            per_game_table = table_container.find('table', id='per_game') if table_container else None
            
            if per_game_table:
                print('Successfully fetched {} table'.format(yr))
                data = pd.read_html(team)[5]
                data['Year'] = str(yr)
                df_list.append(data)
                counter += 1

            else:
                print("Failed to fetch data from {}".format(new_team_url))

    print('\n')


final_df = pd.concat(df_list, ignore_index=True)

///// Currently parsing tables for : CONNECTICUT /////


Fetching data from 2024 ... 
Successfully fetched 2024 table
Fetching data from 2023 ... 
Successfully fetched 2023 table
Fetching data from 2022 ... 
Successfully fetched 2022 table
Fetching data from 2021 ... 
Successfully fetched 2021 table
Fetching data from 2020 ... 
Successfully fetched 2020 table
Fetching data from 2019 ... 
Successfully fetched 2019 table
Fetching data from 2018 ... 
Successfully fetched 2018 table
Fetching data from 2017 ... 
Successfully fetched 2017 table
Fetching data from 2016 ... 
Successfully fetched 2016 table
Fetching data from 2015 ... 
Successfully fetched 2015 table
Fetching data from 2014 ... 
Successfully fetched 2014 table
Fetching data from 2013 ... 
Successfully fetched 2013 table
Fetching data from 2012 ... 
Successfully fetched 2012 table
Fetching data from 2011 ... 
Successfully fetched 2011 table
Fetching data from 2010 ... 
Successfully fetched 2010 table


///// Currently parsing ta

### Analysis

In [10]:
final_df.to_csv('per_game_data.csv')